In [1]:
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
inputfile = "../Examples/c2h2f2/cfour_c2h2f2/output"
# Constants
BOHR_TO_ANG = 0.529177210903
ANG_TO_BOHR = 1. / BOHR_TO_ANG  # 1.889725989      # conversion from Angstrom to bohr
U_TO_AMU = 1. / 5.4857990943e-4            # conversion from g/mol to amu

In [3]:
"""

The original program was written by Dr. A.B. Trofimov. I have modified it a bit,
but its core/engine is original. Also Horst was involved in creating the program, so you can ask him
about the factors.

"""

'\n\nThe original program was written by Dr. A.B. Trofimov. I have modified it a bit,\nbut its core/engine is original. Also Horst was involved in creating the program, so you can ask him\nabout the factors.\n\n'

In [4]:
def parse_coordinates_and_masses(file_path):
    start_reading_coords = False
    start_reading_masses = False
    coordinates = []
    atoms = []
    masses = []

    with open(file_path, 'r') as file:
        for line in file:
            if "Coordinates (in bohr)" in line:
                start_reading_coords = True
                start_reading_masses = False
                # Skip the next two header lines
                next(file)
                next(file)
                continue
            
            if "masses used" in line:
                start_reading_masses = True
                start_reading_coords = False
                continue

            if start_reading_coords:
                if line.strip() == "" or "-----" in line:  # End of the coordinate section
                    start_reading_coords = False
                    continue
                parts = line.split()
                if len(parts) >= 5:
                    atom = parts[0]
                    x = float(parts[2])
                    y = float(parts[3])
                    z = float(parts[4])
                    atoms.append(atom)
                    coordinates.append([x, y, z])

            if start_reading_masses:
                if "Normal Coordinate Analysis" in line:  # End of the masses section
                    start_reading_masses = False
                    continue
                parts = line.split()
                for mass in parts:
                    masses.append(float(mass))

    coordinates_array = np.array(coordinates)
    atoms_array = np.array(atoms)
    masses_array = np.array(masses)
    return atoms_array, coordinates_array, masses_array

# Example usage
file_path = 'output.txt'
atoms_array, coordinates_array, masses_array = parse_coordinates_and_masses(inputfile)
print("Atoms:", atoms_array)
print("Coordinates:", coordinates_array)
print("Masses:", masses_array)

Atoms: ['C' 'C' 'H' 'H' 'F' 'F']
Coordinates: [[-0.          0.          2.77325262]
 [-0.          0.          0.27041081]
 [-0.          1.77252639  3.77410851]
 [-0.         -1.77252639  3.77410851]
 [-0.          2.04009276 -1.16144612]
 [ 0.         -2.04009276 -1.16144612]]
Masses: [12.         12.          1.00782503  1.00782503 18.99840322 18.99840322]


In [5]:
def parse_normal_coordinates(file_path, natoms, is_linear):
    start_reading = False
    frequencies = []
    full_normal_modes = []
    last_line_nmodes = []
    irreps = []
    num_modes = 3 * natoms - 5 if is_linear else 3 * natoms - 6

    with open(file_path, 'r') as file:
        for line in file:
            if "Normal Coordinates" in line:
                start_reading = True
                # next(file)  # Skip the empty line following "Normal Coordinates"
                continue
            
            if start_reading:
                full_blocks = num_modes // 3  # The modes are distributed in 3 columns
                last_block = None if (num_modes % 3) == 0 else (num_modes % 3)
                for _ in range(full_blocks):
                    irreps.extend(next(file).split())
                    freqs = [float(f) for f in next(file).split()]
                    frequencies.extend(freqs)
                    next(file)  # Skip the "VIBRATION" line

                    for _ in range(natoms):
                        line_nmode = next(file).split()[1:]  # Skip the atom label
                        line_nmode = [float(f) for f in line_nmode]
                        full_normal_modes.append(line_nmode)
                    
                    next(file)  # Skip the blank line at the end of the block
                full_normal_modes = np.array(full_normal_modes).reshape(full_blocks,9 * natoms) # 9 because there are 3nmodes in cartesian(xyz) per blockfull_blocks
                full_normal_modes = np.reshape(full_normal_modes, (full_blocks, natoms * 3, 3)) # making the blocks simpler
                
                if last_block:
                    irreps.extend(next(file).split())
                    freqs = [float(f) for f in next(file).split()]
                    frequencies.extend(freqs)
                    next(file)  # Skip the "VIBRATION" line

                    for _ in range(natoms):
                        line_nmode = next(file).split()[1:]  # Skip the atom label
                        line_nmode = [float(f) for f in line_nmode]
                        last_line_nmodes.append(line_nmode)
                    last_line_nmodes = np.array(last_line_nmodes).reshape(-1) # all data in a line
                    last_line_nmodes = np.reshape(last_line_nmodes,(natoms * last_block,3)) # Same structure as before 

                start_reading = False
    arr_nmodes_reshap = []
    for block in range(full_blocks):
        arr_full_lines = [[] for _ in range(3)]
        for idx in range(natoms):
            arr_full_lines[0].append(full_normal_modes[block][0+idx*3])
            arr_full_lines[1].append(full_normal_modes[block][1+idx*3])
            arr_full_lines[2].append(full_normal_modes[block][2+idx*3])
        arr_full_lines = np.array(arr_full_lines)
        arr_nmodes_reshap.extend(arr_full_lines)
    if last_block:
        arr_last_lines = [[] for _ in range(last_block)]
        if last_block == 2:
            for idx in range(natoms):
                print(last_line_nmodes[0+idx*2])
                arr_last_lines[0].append(last_line_nmodes[0+idx*2])
                arr_last_lines[1].append(last_line_nmodes[1+idx*2])
        else:
            for idx in range(natoms):
                arr_last_lines[0].append(last_line_nmodes[0+idx])
        arr_nmodes_reshap.extend(arr_last_lines)
    arr_nmodes_reshap = np.array(arr_nmodes_reshap)

    frequencies_array = np.array(frequencies)
    normal_modes_array = np.array(arr_nmodes_reshap)
    irreps_array = np.array(irreps)
    
    return irreps_array, frequencies_array, normal_modes_array

natoms = 6 
is_linear = False 

irreps_array, frequencies_array, normal_modes_array = parse_normal_coordinates(inputfile, natoms, is_linear)
print("Irreducible Representations:", irreps_array)
print("Frequencies:", frequencies_array)
print("Normal Modes:", normal_modes_array)
np.shape(normal_modes_array)


Irreducible Representations: ['B2' 'A1' 'B1' 'A2' 'B1' 'A1' 'B2' 'B2' 'A1' 'A1' 'A1' 'B2']
Frequencies: [ 437.65  554.94  624.51  723.68  815.49  942.41  972.66 1354.07 1422.52
 1778.08 3210.23 3316.54]
Normal Modes: [[[-0.000e+00  3.903e-01 -0.000e+00]
  [ 0.000e+00 -3.060e-01 -0.000e+00]
  [-0.000e+00  2.895e-01 -3.049e-01]
  [ 0.000e+00  2.895e-01  3.049e-01]
  [ 0.000e+00 -1.002e-01  4.361e-01]
  [-0.000e+00 -1.002e-01 -4.361e-01]]

 [[ 0.000e+00  0.000e+00  3.515e-01]
  [ 0.000e+00 -0.000e+00  2.888e-01]
  [ 0.000e+00 -1.000e-03  1.064e-01]
  [-0.000e+00  1.000e-03  1.064e-01]
  [-0.000e+00 -5.544e-01 -2.790e-01]
  [ 0.000e+00  5.544e-01 -2.790e-01]]

 [[ 2.109e-01 -0.000e+00  0.000e+00]
  [-8.923e-01 -0.000e+00  0.000e+00]
  [ 1.566e-01  0.000e+00  0.000e+00]
  [ 1.566e-01  0.000e+00  0.000e+00]
  [ 2.347e-01 -0.000e+00 -0.000e+00]
  [ 2.347e-01  0.000e+00 -0.000e+00]]

 [[-0.000e+00 -0.000e+00 -0.000e+00]
  [ 0.000e+00 -0.000e+00 -0.000e+00]
  [ 6.930e-01 -0.000e+00 -0.000e+00]


(12, 6, 3)

In [6]:
# Transform coordinates from Bohr to Angtrom
ref_geom = coordinates_array * BOHR_TO_ANG 
ref_geom

array([[-0.        ,  0.        ,  1.46754209],
       [-0.        ,  0.        ,  0.14309524],
       [-0.        ,  0.93798057,  1.99717221],
       [-0.        , -0.93798057,  1.99717221],
       [-0.        ,  1.0795706 , -0.61461082],
       [ 0.        , -1.0795706 , -0.61461082]])

## Calculation of the reduced mass of each mode

In [7]:
"""
205 c    Calculating reduced mass
206 c
207       do 40 i=1,nummod
208          Summ=zero
209          mm=1
210          do 30 j=1,natoms*3
211          xmasa=XMass(mm)
212          Summ=Summ+zmod(j,i)*zmod(j,i)/xmasa
213          If (mod(j,3) .eq. 0) mm=mm+1
214    30    Continue
215 c
216       Redm(i)=one/Summ
"""

red_mass = []
for mode in normal_modes_array:
    suma = np.sum(mode**2 /masses_array[:,np.newaxis])
    red_mass.append(1/suma)
red_mass = np.array(red_mass)
red_mass


array([ 2.54855714, 12.45866919,  8.03073359,  1.04707083,  1.30137166,
        6.65826092,  1.71659702,  7.30098322,  1.16622613,  7.62858541,
        1.05974663,  1.12100992])

## CFOUR normal modes to Gaussian normal modes

In [8]:
"""
220 c   l(aces) to l(Gauss)
221 c
222       do 50 i=1,nummod
223          RedSq=dsqrt(Redm(i))
224          mm=1
225          do 50 j=1,natoms*3
226          xmasa=XMass(mm)
227          zmod(j,i)=zmod(j,i)*RedSq/dsqrt(xmasa)
228          If (mod(j,3) .eq. 0) mm=mm+1
229    50 Continue


"""
new_modes = np.empty_like(normal_modes_array)
red_mass_sqrt = np.sqrt(red_mass)

for idx, mode in enumerate(normal_modes_array):
    new_modes[idx] = mode * red_mass_sqrt[idx] / np.sqrt(masses_array[:,np.newaxis])

print(normal_modes_array[0])
print(new_modes[0])


[[-0.      0.3903 -0.    ]
 [ 0.     -0.306  -0.    ]
 [-0.      0.2895 -0.3049]
 [ 0.      0.2895  0.3049]
 [ 0.     -0.1002  0.4361]
 [-0.     -0.1002 -0.4361]]
[[-0.          0.1798685  -0.        ]
 [ 0.         -0.14101912 -0.        ]
 [-0.          0.46036594 -0.48485518]
 [ 0.          0.46036594  0.48485518]
 [ 0.         -0.03669918  0.15972567]
 [-0.         -0.03669918 -0.15972567]]


## Calculation of dimensionless normal mode displacements

In [9]:
"""
 18       data c1 / 219480.65050000d+00 /
 19       data c2 /      6.42297000d-02 /
 20       data c3 /      5.48580260d-04 /
 21       data c4 /      5.29177060d-01 /
 22 c
 23 c---  Compute some conversion factors
 24 c
 25       c1c2c3 = c1 * dsqrt (c2 * c3)
 26       c1c3c4 = c4 * dsqrt (c1 * c3)

"""
# Defining constants
# Hartree to cm-1
HARTREE_TO_CM1 = 219474.63136320
# c2 is force related
AMU_TO_AU =  5.485799090441e-4  
BOHR_TO_ANG = 0.529177210903

c1c3c4 = BOHR_TO_ANG * np.sqrt(HARTREE_TO_CM1* AMU_TO_AU)

In [10]:
"""
145 c---  Check for consistency and
146 c---  compute scaling factor
147 c
148       do 90 j=1,nmodes
149 c
150 c---     compute for each mode:
151 c---     a) Norm
152 c---     b) Reduced Mass
153 c---     c) Frequency
154 c---     d) Scaling factor to get unit 
155 c---        normal coordinate displacement
156 c
157          sum1 = zero
158          sum2 = zero
159 c
160          do 80 i=1,natom3
161             term = zmod (i,j)
162             terx = term * term
163             sum1 = sum1 + terx
164             sum2 = sum2 + terx * amuc (i)
165    80    continue
166 c
167          znorm (j) = sum1
168          zred  (j) = sum2
169 c        zfreq (j) = c1c2c3 * dsqrt ( force  (j) / redmas (j) )
170          runit (j) = c1c3c4 / dsqrt ( redmas (j) * freq   (j) )
171    90 continue
172 c

"""
znorm = np.empty_like(red_mass)
zred  = np.empty_like(red_mass)
runit = np.empty_like(red_mass)

for idx, mode in enumerate(new_modes):
    mode2 = mode * mode
    znorm[idx] = np.sum(mode2)
    zred[idx]  = np.sum(mode2 * masses_array[:,np.newaxis])

# znorm should give an array filled with ones 
# zred should give an array filled with the reduced masses

print(znorm)
print(zred)
runit = c1c3c4 / np.sqrt(red_mass * frequencies_array)


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ 2.54846822 12.45869822  8.02987752  1.04559028  1.30152682  6.65874032
  1.71670178  7.30115516  1.16616871  7.6278066   1.05980832  1.1208771 ]


In [11]:
print(np.shape(new_modes)[0])
print(np.shape(red_mass))
znorm = np.zeros(new_modes.shape[0])
print(znorm)

12
(12,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [12]:
"""
238 c---  Apply scaling factor: modify normal modes
239 c
240       do 120 i=1,natom3
241       do 120 j=1,nmodes
242          zmod (i,j) = zmod (i,j) * runit (j)
243   120 continue
244 c
"""

displacements = new_modes * runit[:,np.newaxis, np.newaxis]
displacements

array([[[-0.00000000e+00,  3.12721477e-02, -0.00000000e+00],
        [ 0.00000000e+00, -2.45177484e-02, -0.00000000e+00],
        [-0.00000000e+00,  8.00397598e-02, -8.42974880e-02],
        [ 0.00000000e+00,  8.00397598e-02,  8.42974880e-02],
        [ 0.00000000e+00, -6.38056226e-03,  2.77700918e-02],
        [-0.00000000e+00, -6.38056226e-03, -2.77700918e-02]],

       [[ 0.00000000e+00,  0.00000000e+00,  2.50106476e-02],
        [ 0.00000000e+00, -0.00000000e+00,  2.05492889e-02],
        [ 0.00000000e+00, -2.45526097e-04,  2.61239767e-02],
        [-0.00000000e+00,  2.45526097e-04,  2.61239767e-02],
        [-0.00000000e+00, -3.13512530e-02, -1.57774163e-02],
        [ 0.00000000e+00,  3.13512530e-02, -1.57774163e-02]],

       [[ 1.41458648e-02, -0.00000000e+00,  0.00000000e+00],
        [-5.98499535e-02, -0.00000000e+00,  0.00000000e+00],
        [ 3.62445519e-02,  0.00000000e+00,  0.00000000e+00],
        [ 3.62445519e-02,  0.00000000e+00,  0.00000000e+00],
        [ 1.25111747

In [13]:
"""
283      #('*Distorted Geometries: x,y,z-coordinates (A)*',0,2,1,0,1h-)
284 c
285       do 170 j=1,nmodes
286          k = 0
287          do 140 i=1,natoms
288 c
289             xyzpls (i,1) = xyz (i,1) + zmod (k+1,j) * factor
290             xyzpls (i,2) = xyz (i,2) + zmod (k+2,j) * factor
291             xyzpls (i,3) = xyz (i,3) + zmod (k+3,j) * factor
292 c
293             xyzmns (i,1) = xyz (i,1) - zmod (k+1,j) * factor
294             xyzmns (i,2) = xyz (i,2) - zmod (k+2,j) * factor
295             xyzmns (i,3) = xyz (i,3) - zmod (k+3,j) * factor
296 c
297             k = k + 3
298   140    continue

"""

"\n283      #('*Distorted Geometries: x,y,z-coordinates (A)*',0,2,1,0,1h-)\n284 c\n285       do 170 j=1,nmodes\n286          k = 0\n287          do 140 i=1,natoms\n288 c\n289             xyzpls (i,1) = xyz (i,1) + zmod (k+1,j) * factor\n290             xyzpls (i,2) = xyz (i,2) + zmod (k+2,j) * factor\n291             xyzpls (i,3) = xyz (i,3) + zmod (k+3,j) * factor\n292 c\n293             xyzmns (i,1) = xyz (i,1) - zmod (k+1,j) * factor\n294             xyzmns (i,2) = xyz (i,2) - zmod (k+2,j) * factor\n295             xyzmns (i,3) = xyz (i,3) - zmod (k+3,j) * factor\n296 c\n297             k = k + 3\n298   140    continue\n\n"